# Шаг 1
### Подключение необходимых библиотек

In [ ]:
import math
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from scipy.special import comb, gamma
import scipy.stats as stats
